In [1]:
import pandas as pd

1 = spam
0 = no spam

In [2]:
df = pd.read_csv('train.csv')
df.head()

,sms,label
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...\n,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [23]:
df.shape

(5574, 3)

In [4]:
from transformers import BertTokenizer, BertModel
import torch

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Apply BERT embeddings

*Key Points:*


Padding: If message is shorter than 128 tokens, padding will add special tokens (e.g., [PAD]) to make it exactly 128 tokens long. This ensures that all inputs have the same length.

Truncation: If your message is longer than 128 tokens, it will be cut off at 128 tokens. So, yes, if your message is long and important information falls beyond the 128th token, it will be lost.

#### set padding to 256 given that 128 doesn't fit well the data

In [5]:
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        outputs = model(**inputs)
    # Get the [CLS] token embedding
    return outputs.pooler_output.squeeze().numpy()


In [6]:
from tqdm import tqdm
tqdm.pandas()

# Apply BERT to each row and create a new column with the embeddings
df['bert_embedding'] = df['sms'].progress_apply(get_bert_embedding)


100%|██████████████████████████████████████████████████████████████████████████████| 5574/5574 [03:52<00:00, 23.96it/s]


In [8]:
df.head()

,sms,label,bert_embedding
0,"Go until jurong point, crazy.. Available only ...",0,"[-0.69134617, -0.38095, -0.95244503, 0.7509417..."
1,Ok lar... Joking wif u oni...\n,0,"[-0.8942947, -0.6027674, -0.9841888, 0.9353531..."
2,Free entry in 2 a wkly comp to win FA Cup fina...,1,"[-0.7034378, -0.50361043, -0.9644544, 0.636864..."
3,U dun say so early hor... U c already then say...,0,"[-0.7182437, -0.3971464, -0.7705447, 0.7624593..."
4,"Nah I don't think he goes to usf, he lives aro...",0,"[-0.78104824, -0.57640296, -0.9893936, 0.81220..."


# Training 

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn import datasets

In [10]:
X = np.vstack(df['bert_embedding'].values)
y = df['label'].to_numpy()                  # Convert labels to a NumPy array

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## SVM model

In [12]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

SVC(kernel='linear')

In [13]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = svm_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9901345291479821

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       954
           1       0.97      0.96      0.97       161

    accuracy                           0.99      1115
   macro avg       0.98      0.98      0.98      1115
weighted avg       0.99      0.99      0.99      1115



##  Gaussian Naive Bayes classifier

In [14]:
from sklearn.naive_bayes import GaussianNB

# Create a Gaussian Naive Bayes classifier
model = GaussianNB()

# Fit the model to your data
model.fit(X, y)

# Make predictions
y_pred = model.predict(X)

In [15]:
# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y, y_pred))
print("Classification Report:\n", classification_report(y, y_pred))

Accuracy: 0.5055615357014711
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.43      0.60      4827
           1       0.21      0.97      0.34       747

    accuracy                           0.51      5574
   macro avg       0.60      0.70      0.47      5574
weighted avg       0.88      0.51      0.57      5574



## DecisionTreeClassifier

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [17]:
# Create and train the Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)


DecisionTreeClassifier(random_state=42)

In [18]:
# Make predictions
y_pred = dt_model.predict(X_test)
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Print classification report for more details
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9587443946188341
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       954
           1       0.85      0.86      0.86       161

    accuracy                           0.96      1115
   macro avg       0.91      0.92      0.92      1115
weighted avg       0.96      0.96      0.96      1115



## RandomForestClassifier

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
# Create a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [21]:
# Train the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [22]:
# Make predictions
y_pred = rf_model.predict(X_test)
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Print classification report for more details
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.9730941704035875
Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       954
           1       0.99      0.83      0.90       161

    accuracy                           0.97      1115
   macro avg       0.98      0.91      0.94      1115
weighted avg       0.97      0.97      0.97      1115

